In [1]:

import c302
from c302.ConnectomeReader import analyse_connections
from c302.ConnectomeReader import check_neurons

all_data = {}
all_data["Values"] =["Num neurons", 
                     "Missing neurons", 
                     "Non neurons", 
                     "Num muscles", 
                     "Num N->N conns", 
                     "Num N with ->M", 
                     "Num N->M conns", 
                     "N->N neurotrans.", 
                     "N->M neurotrans."]

readers = {"SSData": "c302.SpreadsheetDataReader", 
           "UpdSSData": "c302.UpdatedSpreadsheetDataReader",
           "UpdSSData2": "c302.UpdatedSpreadsheetDataReader2",
           "OpenWorm": "c302.OpenWormReader",
           "White_A": "c302.White_A", 
           "White_L4": "c302.White_L4", 
           "White_whole": "c302.White_whole",
           "Varshney": "c302.VarshneyDataReader",
           "Witvliet1": "c302.WitvlietDataReader1",
           "Witvliet2": "c302.WitvlietDataReader2",
           "WormNeuroAtlas": "c302.WormNeuroAtlasReader",
          }

def shorten_neurotransmitter(nt):
    return nt.replace('Acetylcholine', 'ACh').replace('Serotonin', '5HT').replace('Glutamate', 'Glu')\
             .replace('Tyramine', 'Tyr').replace('FMRFamide','FMRFam').replace('Generic_', 'Gen_')

for name, reader in readers.items():

    print("\n****** Importing dataset %s using %s ******"% (name, reader))
    

    exec("from %s import read_data, read_muscle_data"%reader)
    cells, neuron_conns = read_data(include_nonconnected_cells=True)

    preferred, not_in_preferred, missing_preferred = check_neurons(cells)

    neuron_nts = {}
    for c in neuron_conns:
        nt = c.synclass
        if len(nt)==0: nt='**MISSING**'

        if not nt in neuron_nts:
            neuron_nts[nt] = 0
        
        neuron_nts[nt] +=1
    
    nts_info = ''
    for nt in sorted(neuron_nts.keys()):
        nts_info+='%s (%i)<br/>'%(shorten_neurotransmitter(nt), neuron_nts[nt])

 
    neurons2muscles, muscles, muscle_conns = read_muscle_data()
    
    muscle_nts = {}
    for c in muscle_conns:
        nt = c.synclass
        if len(nt)==0: nt='**MISSING**'

        if not nt in muscle_nts:
            muscle_nts[nt] = 0
        
        muscle_nts[nt] +=1
    
    m_nts_info = ''
    for nt in sorted(muscle_nts):
        m_nts_info+='%s (%i)<br/>'%(shorten_neurotransmitter(nt), muscle_nts[nt])


    all_data[name] =[len(preferred),
                     len(missing_preferred), 
                     len(not_in_preferred), 
                     len(muscles), 
                     len(neuron_conns), 
                     len(neurons2muscles), 
                     len(muscle_conns),
                     nts_info,
                     m_nts_info]

print('\nFinished loading all the data from the readers!')


****** Importing dataset SSData using c302.SpreadsheetDataReader ******
c302      >>>  Opened Excel file: /opt/homebrew/anaconda3/envs/py39n/lib/python3.9/site-packages/c302/data/CElegansNeuronTables.xls
c302      >>>  Opened Excel file: /opt/homebrew/anaconda3/envs/py39n/lib/python3.9/site-packages/c302/data/CElegansNeuronTables.xls

****** Importing dataset UpdSSData using c302.UpdatedSpreadsheetDataReader ******
c302      >>>  Opened file: /opt/homebrew/anaconda3/envs/py39n/lib/python3.9/site-packages/c302/data/herm_full_edgelist.csv
c302      >>>  Opened file: /opt/homebrew/anaconda3/envs/py39n/lib/python3.9/site-packages/c302/data/herm_full_edgelist.csv

****** Importing dataset UpdSSData2 using c302.UpdatedSpreadsheetDataReader2 ******
c302      >>>  Opened file: /opt/homebrew/anaconda3/envs/py39n/lib/python3.9/site-packages/c302/data/herm_full_edgelist_MODIFIED.csv
c302      >>>  Opened file: /opt/homebrew/anaconda3/envs/py39n/lib/python3.9/site-packages/c302/data/herm_full_edg

/opt/homebrew/anaconda3/envs/py39n/lib/python3.9/site-packages/wormneuroatlas/WormBase.py:52: UserWarning: Wormbase.org updated its database to a new version (WS291). The version of Worm Neuro Atlas that you are using has been built for the wormbase database version WS287. To ensure reproducible results, upgrade Worm Neuro Atlas with `python -m pip install --upgrade wormneuroatlas` If this warning persists after upgrading, let the developers know by opening an issue here: https://github.com/francescorandi/wormneuroatlas/issues. NOTE: You can still use Worm Neuro Atlas in the meantime. The metadata accessible via  wormneuroatlas.WormBase.get_metadata() and wormneuroatlas.NeuroAtlas.get_metadata() contain the version of wormbase that you are currently using, so make sure you save the metadata alongside your results. 
  warnings.warn(w)


c302      >>>  Initialising WormNeuroAtlasReader

Finished loading all the data from the readers!


In [2]:
import pandas as pd
import numpy as np
from IPython.display import HTML

df_all = pd.DataFrame(all_data)
df_all.set_index("Values")

h = HTML(df_all.to_html(escape=False, index=False));h

Values,SSData,UpdSSData,UpdSSData2,OpenWorm,White_A,White_L4,White_whole,Varshney,Witvliet1,Witvliet2,WormNeuroAtlas
Num neurons,302,302,302,302,182,180,301,279,181,180,300
Missing neurons,0,0,0,0,120,122,1,23,121,122,2
Non neurons,0,0,0,0,9,5,8,0,9,7,0
Num muscles,97,95,95,95,32,32,1,1,32,32,0
Num N->N conns,3363,5806,5821,5805,1694,1593,2847,6264,2251,2274,5701
Num N with ->M,127,254,254,159,38,38,114,115,58,47,0
Num N->M conns,564,1118,1118,926,205,176,114,153,230,216,0
N->N neurotrans.,ACh (394)ACh_Tyr (29)Dopamine (120)FMRFam (247)GABA (200)Gen_GJ (1084)Glu (962)Octapamine (20)5HT (178)5HT_ACh (104)5HT_Glu (25),ACh (3581)GABA (57)Gen_GJ (2168),ACh (3596)GABA (57)Gen_GJ (2168),ACh (2008)ACh_GJ (222)ACh_Tyr (29)ACh_Tyr_GJ (13)Dopamine (98)Dopamine_GJ (13)FMRFam (170)FMRFam_GJ (90)FRMFemide (45)FRMFemide_GJ (33)GABA (182)GABA_GJ (76)Gen_GJ (1356)Glu (824)Glu_GJ (324)Octapamine (19)Octapamine_GJ (4)5HT (163)5HT_ACh (91)5HT_ACh_GJ (13)5HT_GJ (20)5HT_Glu (11)5HT_Glu_GJ (1),ACh (1424)Gen_GJ (270),ACh (1304)Gen_GJ (289),ACh (2237)GABA (35)Gen_GJ (575),Gen_CS (5233)Gen_GJ (1031),Gen_CS (1979)Gen_GJ (272),Gen_CS (1970)Gen_GJ (304),ACh (2017)GABA (631)Gen_CS (431)Gen_GJ (1650)Glu (972)
N->M neurotrans.,**MISSING** (2)ACh (377)AChplus_Tyr (6)Dopamine (1)FMRFam (3)FRMFemide (5)GABA (126)Glu (35)5HT (1)5HTplus_ACh (8),ACh (808)GABA (116)Gen_GJ (194),ACh (808)GABA (116)Gen_GJ (194),ACh (924)Gen_GJ (2),ACh (205),ACh (176),ACh (96)GABA (18),Gen_CS (153),Gen_CS (230),Gen_CS (216),


In [5]:

from neuroml.loaders import read_neuroml2_file
    
parameter_set = 'C1'

from c302.c302_Oscillator import setup
from c302.c302_utils import generate_conn_matrix

import matplotlib.pyplot as plt

plt.ioff()

for reader_module in readers.values():
    
    reader = reader_module[5:]
        
    print("\n=== Generating using reader %s"%reader)
    setup(parameter_set, 
          generate=True, 
          target_directory='images',
          data_reader=reader,
          verbose=False)

    nml_filename = 'c302_%s_Oscillator.net.nml'%parameter_set

    nml_doc = read_neuroml2_file('images/%s'%nml_filename)
    nml_doc.networks[0].id = reader.replace('Reader','').replace('Spreadsheet','SS').replace('Updated','Upd')

    generate_conn_matrix(nml_doc, save_fig_dir='./images', figsize=(6,6))
        
print('Finished generation of images!')



=== Generating using reader SpreadsheetDataReader
c302      >>>  Set default parameters for C
c302      >>>  Set default parameters for C
c302      >>>  Set default parameters for C
c302      >>>  Set default parameters for C1
c302      >>>  Opened Excel file: /opt/homebrew/anaconda3/envs/py39n/lib/python3.9/site-packages/c302/data/CElegansNeuronTables.xls
c302      >>>  Opened Excel file: /opt/homebrew/anaconda3/envs/py39n/lib/python3.9/site-packages/c302/data/CElegansNeuronTables.xls
c302      >>>  Writing generated network to: /Users/padraig/c302/examples/test/images/c302_C1_Oscillator.net.nml
c302      >>>  Saving connectivity figure to: ./images/SSData_exc_to_neurons.png
c302      >>>  Saving connectivity figure to: ./images/SSData_inh_to_neurons.png
c302      >>>  Saving connectivity figure to: ./images/SSData_elec_neurons_neurons.png

=== Generating using reader UpdatedSpreadsheetDataReader
c302      >>>  Set default parameters for C
c302      >>>  Set default parameters for C


|EXC|INH|GJ|
|-|-|-|
|![exc](./images/SSData_exc_to_neurons.png)|![inh](./images/SSData_inh_to_neurons.png)|![gj](./images/SSData_elec_neurons_neurons.png)|
|![exc](./images/UpdSSData_exc_to_neurons.png)|![inh](./images/UpdSSData_inh_to_neurons.png)|![gj](./images/UpdSSData_elec_neurons_neurons.png)|
|![exc](./images/UpdSSData2_exc_to_neurons.png)|![inh](./images/UpdSSData2_inh_to_neurons.png)|![gj](./images/UpdSSData2_elec_neurons_neurons.png)|
|![exc](./images/OpenWorm_exc_to_neurons.png)|![inh](./images/OpenWorm_inh_to_neurons.png)|![gj](./images/OpenWorm_elec_neurons_neurons.png)|
|![exc](./images/White_whole_exc_to_neurons.png)|![inh](./images/White_whole_inh_to_neurons.png)|![gj](./images/White_whole_elec_neurons_neurons.png)|
|![exc](./images/VarshneyData_exc_to_neurons.png)|![inh](./images/VarshneyData_inh_to_neurons.png)|![gj](./images/VarshneyData_elec_neurons_neurons.png)|
|![exc](./images/WitvlietData2_exc_to_neurons.png)|![inh](./images/WitvlietData2_inh_to_neurons.png)|![gj](./images/WitvlietData2_elec_neurons_neurons.png)|
|![exc](./images/WormNeuroAtlas_exc_to_neurons.png)|![inh](./images/WormNeuroAtlas_inh_to_neurons.png)|![gj](./images/WormNeuroAtlas_elec_neurons_neurons.png)|

